# Filtragem Baseada em Conteudo
## Linear

In [46]:
import pandas as pd
import numpy as np
import math

import time

In [47]:
# Importando .csv
train = pd.read_csv('dataset/train_data.csv', header=None,  skiprows=[0], usecols=[0,1,2])

# Obtendo generos dos filmes
m = pd.read_csv('dataset/movies_data.csv')
genres = pd.get_dummies(m.set_index(['movie_id']).genres.str.split('|', expand=True).stack(dropna=False)).sum(level=0)

In [48]:
# Função para cria modelo
def fbc_linear(train, test, features, lr = 0.05, reg = 0.002, miter = 10):
    nusers = np.append(train[:,0], test[:,0]).max()
    nitems = np.append(train[:,1], test[:,1]).max()
    nfeatures = len(features[1])-1
    # features = np.hstack((features,np.ones((len(features),1))))
    profiles = np.random.normal(loc = 0, scale = 0.1, size=(nusers, nfeatures+1))
    error = list()
    for l in range(0, miter):
        sq_error = 0
        for j in range(0, len(train)):
            u = train[j, 0]-1
            i = train[j, 1]-1
            r_ui = train[j, 2]-1
            e_ui = np.dot(profiles[u, ], features[i, ]) - r_ui
            sq_error += e_ui**2
            for k in range(nfeatures-1):
                profiles[u, k] = profiles[u, k] - lr * (e_ui * features[i, k] + reg * profiles[u, k])
            k = nfeatures
            profiles[u, k] = profiles[u, k] - lr * (e_ui * features[i, k])
        error.append(math.sqrt(sq_error/len(train)))
    return { "profiles": profiles, "error": error }



In [49]:
# Predizendo nota
def predict(model, user, item, features):
    return np.dot(model["profiles"][user-1, ], features[item-1, ])

In [50]:
# Avaliando predições
def rmse(model, test, features):
    sum_err = 0
    for t in test:
        u = t[0]
        i = t[1]
        r_ui = t[2]
        pred = predict(model, u, i, features)
        error = (r_ui - pred)**2
        sum_err += error
    return math.sqrt(sum_err/len(test))

In [51]:
def validation(model, data, features, train_split = 0.75, lr = 0.05, reg = 0.02, miter = 10):
    data = data.sample(frac=1)
    train_len = int(train_split * len(data))
    
    test = data[train_len:].values

    train_len = int(train_split * train_len)
    train = data[:train_len].values
    train_validation = data[train_len:].values

    start_time = time.time()
    model = model(train, train_validation, features, lr, reg, miter)
    print("Tempo de treinamento em segundos: ", time.time() - start_time)
    start_time = time.time()
    print("RMSE :", rmse(model, test, features))
    print("Tempo de predição em segundos: ", time.time() - start_time)

In [52]:
# model = fbc_linear(r.values[:482205], r.values[482205:], genres.values)

In [53]:
# rmse(model, r.values[482205:], genres.values)

In [54]:
features = genres.values
features = np.hstack((features,np.ones((len(features),1))))
validation(fbc_linear, train, features, 0.75, lr = 0.05, reg = 0.02, miter = 10)

Tempo de treinamento em segundos:  102.38923835754395
RMSE : 1.4714520646681342
Tempo de predição em segundos:  0.9514524936676025


## Execução 1 treino 75% lr = 0.05, reg = 0.02, miter = 1
Tempo de treinamento em segundos:  8.421872615814209 <br />
RMSE : 1.535468933491189 <br />
Tempo de predição em segundos:  0.9305610656738281 <br />

## Execução 2 treino 75% lr = 0.05, reg = 0.02, miter = 10
Tempo de treinamento em segundos:  88.66791653633118 <br />
RMSE : 1.4554809871965142 <br />
Tempo de predição em segundos:  0.8296205997467041 <br />

## Execução 3 treino 75% lr = 0.10, reg = 0.05, miter = 10
Tempo de treinamento em segundos:  92.5954806804657 <br />
RMSE : 1.4727703422035867 <br />
Tempo de predição em segundos:  0.9102404117584229 <br />

## Execução 3 treino 75%  lr = 0.10, reg = 0.10, miter = 10
Tempo de treinamento em segundos:  102.38923835754395 <br />
RMSE : 1.4714520646681342 <br />
Tempo de predição em segundos:  0.9514524936676025 <br />